# Trelliscope Viewer Integration

This notebook demonstrates how to use the trelliscope viewer to explore displays interactively.

The viewer provides:
- **Interactive exploration**: View, filter, and sort panels
- **Multi-display structure**: Organized display collections
- **Development server**: Quick preview during development
- **Static export**: Ready for web deployment

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from trelliscope import Display

## 1. Basic Viewer Usage

The simplest way to view a display is using the `.view()` method, which:
1. Writes the display if not already written
2. Starts a local web server
3. Opens the display in your browser
4. Keeps the server running in the background

In [ ]:
# Create sample data
np.random.seed(42)
data = pd.DataFrame({
    'id': range(20),
    'value': np.random.randn(20) * 100 + 500,
    'category': np.random.choice(['A', 'B', 'C'], 20),
    'score': np.random.uniform(0, 100, 20)
})

print(data.head())

In [ ]:
# Create a simple plot function
def make_plot(row):
    fig, ax = plt.subplots(figsize=(6, 4))
    
    # Generate some data based on row values
    x = np.linspace(0, 10, 100)
    y = np.sin(x + row['value'] / 100) * row['score']
    
    ax.plot(x, y, linewidth=2)
    ax.set_title(f"Category {row['category']} - ID {row['id']}")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

In [ ]:
# Create display with plots
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# Generate plots for each row
data['panel'] = data.apply(make_plot, axis=1)

# Create display
display = (
    Display(data, name="basic_viewer_demo", path=output_dir)
    .set_panel_column('panel')
    .infer_metas()
    .set_default_layout(nrow=2, ncol=3)
)

# Write display (returns output path)
output_path = display.write(force=True)
print(f"Display written to: {output_path}")
print(f"\nMulti-display structure:")
print(f"  - config.json: {(output_path / 'config.json').exists()}")
print(f"  - displays/displayList.json: {(output_path / 'displays' / 'displayList.json').exists()}")
print(f"  - displays/basic_viewer_demo/displayInfo.json: {(output_path / 'displays' / 'basic_viewer_demo' / 'displayInfo.json').exists()}")

In [ ]:
# Launch viewer
# This will:
# 1. Use the already-written display
# 2. Start a web server on specified port
# 3. Open your browser to http://localhost:PORT
# 4. Display the interactive viewer

url = display.view(port=6547, open_browser=True)
print(f"Viewer available at: {url}")
print("Server is running in background. Press Ctrl+C in terminal to stop.")

## 2. Understanding the Multi-Display Structure

The viewer uses a multi-display structure that allows multiple displays in one collection:

In [ ]:
# Inspect the generated structure
import json

# Read config.json
with open(output_path / 'config.json') as f:
    config = json.load(f)
    print("config.json:")
    print(json.dumps(config, indent=2))

print("\n" + "="*50 + "\n")

# Read displayList.json
with open(output_path / 'displays' / 'displayList.json') as f:
    display_list = json.load(f)
    print("displays/displayList.json:")
    print(json.dumps(display_list, indent=2))

## 3. Working with Display Configuration

Customize display behavior through panel options and default state:

In [ ]:
# Create display with custom configuration
data_custom = data.copy()
data_custom['panel'] = data_custom.apply(make_plot, axis=1)

display_custom = (
    Display(
        data_custom,
        name="custom_config_demo",
        path=output_dir,
        description="Display with custom configuration"
    )
    .set_panel_column('panel')
    .infer_metas()
    .set_panel_options(width=800, height=600)  # Custom panel dimensions
    .set_default_layout(nrow=2, ncol=2, arrangement="row")
    .set_default_labels(["category", "score"])  # Show these labels on panels
    .add_default_sort("score", "desc")  # Sort by score descending
    .write(force=True)
)

print(f"Custom display created at: {display_custom}")
print(f"\nTo view: display.view(port=6548)")

## 4. Viewing Display Details

Examine the displayInfo.json to see how configuration is stored:

In [ ]:
# Read displayInfo.json
display_info_path = output_path / 'displays' / 'basic_viewer_demo' / 'displayInfo.json'
with open(display_info_path) as f:
    display_info = json.load(f)

# Show key sections
print("Display Name:", display_info['name'])
print("Description:", display_info['description'])
print("Number of panels:", display_info['n'])
print("\nPanel dimensions:")
print(f"  Width: {display_info['width']}")
print(f"  Height: {display_info['height']}")
print("\nDefault layout:")
print(json.dumps(display_info['state']['layout'], indent=2))
print("\nPanel interface:")
print(json.dumps(display_info['panelInterface'], indent=2))
print("\nMeta variables:")
for meta in display_info['metas']:
    print(f"  - {meta['varname']}: {meta['type']}")

## 5. Real-World Example: Time Series Analysis

Let's create a more realistic example with time series data.

In [ ]:
# Generate synthetic time series data
from datetime import datetime, timedelta

np.random.seed(123)
n_series = 30
n_points = 365

# Create metadata for each series
metadata = pd.DataFrame({
    'series_id': range(n_series),
    'region': np.random.choice(['North', 'South', 'East', 'West'], n_series),
    'category': np.random.choice(['Product A', 'Product B', 'Product C'], n_series),
    'mean_value': np.random.uniform(100, 1000, n_series),
    'volatility': np.random.uniform(0.1, 0.5, n_series),
    'trend': np.random.choice(['up', 'down', 'flat'], n_series)
})

print(metadata.head())

In [ ]:
# Function to generate time series plot
def make_time_series_plot(row):
    # Generate data based on metadata
    dates = pd.date_range(start='2023-01-01', periods=n_points, freq='D')
    
    # Base trend
    if row['trend'] == 'up':
        trend = np.linspace(0, row['mean_value'] * 0.5, n_points)
    elif row['trend'] == 'down':
        trend = np.linspace(0, -row['mean_value'] * 0.3, n_points)
    else:
        trend = np.zeros(n_points)
    
    # Add seasonality
    seasonal = np.sin(np.linspace(0, 4*np.pi, n_points)) * row['mean_value'] * 0.2
    
    # Add noise
    noise = np.random.randn(n_points) * row['mean_value'] * row['volatility']
    
    # Combine
    values = row['mean_value'] + trend + seasonal + noise
    
    # Create plot
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(dates, values, linewidth=1.5, alpha=0.8)
    ax.set_title(f"{row['region']} - {row['category']}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Value")
    ax.grid(True, alpha=0.3)
    
    # Add trend line
    z = np.polyfit(range(n_points), values, 1)
    p = np.poly1d(z)
    ax.plot(dates, p(range(n_points)), "r--", alpha=0.5, linewidth=2, label='Trend')
    ax.legend()
    
    plt.tight_layout()
    return fig

In [ ]:
# Generate plots for time series
metadata['panel'] = metadata.apply(make_time_series_plot, axis=1)

ts_display = (
    Display(
        metadata,
        name="time_series_analysis",
        path=output_dir,
        description="Time series analysis by region and category"
    )
    .set_panel_column('panel')
    .infer_metas()
    .set_default_layout(nrow=2, ncol=2)
    .set_default_labels(["region", "category", "trend"])
    .add_default_sort("mean_value", "desc")
    .write(force=True)
)

print(f"Time series display created with {len(metadata)} panels")
print(f"Output: {ts_display}")

In [ ]:
# View the time series display
# ts_display.view(port=6549, open_browser=True)
print("To view the time series display, uncomment the line above or run:")
print("  ts_display.view(port=6549)")

## 6. Accessing Displays via HTTP

When the server is running, you can access files directly:

In [ ]:
print("Multi-display structure URLs (when server running on port 6547):")
print()
print("Root files:")
print("  http://localhost:6547/index.html")
print("  http://localhost:6547/config.json")
print()
print("Display list:")
print("  http://localhost:6547/displays/displayList.json")
print()
print("Display configuration:")
print("  http://localhost:6547/displays/basic_viewer_demo/displayInfo.json")
print()
print("Panel images:")
print("  http://localhost:6547/displays/basic_viewer_demo/panels/0.png")
print("  http://localhost:6547/displays/basic_viewer_demo/panels/1.png")

## 7. Tips and Best Practices

### Development Workflow
1. **During development**: Use `.write()` to generate files, then `.view()` for quick iteration
2. **For sharing**: Deploy the generated multi-display structure to web hosting
3. **Testing**: Check that all required files exist before deploying

### Performance
- For large displays (>100 panels), consider using lower resolution images
- Use `set_panel_options(width=, height=)` to control panel sizing
- Initial sorts can help users navigate large displays

### Deployment
- The entire output directory can be deployed to static hosting (GitHub Pages, Netlify, etc.)
- Test on multiple browsers
- Use specific viewer versions for production (currently using 0.7.16)

### Configuration
- Use `set_default_labels()` to show important metadata on panels
- Use `add_default_sort()` to highlight important panels first
- Use `set_default_layout()` to control grid arrangement

## Summary

This notebook demonstrated:

1. **Basic viewing**: Write and launch interactive viewer with `.write()` and `.view()`
2. **Multi-display structure**: Understanding config.json, displayList.json, and display directories
3. **Configuration**: Customize panel options, layout, labels, and sorting
4. **Display inspection**: Examine generated JSON files
5. **Real-world example**: Time series analysis workflow

The trelliscope viewer makes it easy to explore large collections of visualizations interactively!